In [1]:
import os
import zipfile
import tensorflow as tf

# Download the dataset from Kaggle and save in Google Colab Files
!kaggle datasets download -d prasunroy/natural-images

# Unzip the natural-images.zip
with zipfile.ZipFile('natural-images.zip', 'r') as zip_ref:
  zip_ref.extractall('')

# Define path for original dataset
data_dir = 'natural_images'

Dataset URL: https://www.kaggle.com/datasets/prasunroy/natural-images
License(s): CC-BY-NC-SA-4.0
 99% 340M/342M [00:14<00:00, 28.4MB/s]
100% 342M/342M [00:14<00:00, 25.2MB/s]


In [2]:
import shutil
import random

# Define path for split data
split_dir = 'data_split'

# Define path for train, validation and test directory
train_dir = os.path.join(split_dir, 'train')
val_dir = os.path.join(split_dir, 'val')
test_dir = os.path.join(split_dir, 'test')

# Create directories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Split ratio
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Split data
categories = os.listdir(data_dir)
seed = 42

for category in categories:
  category_path = os.path.join(data_dir, category)

  if not os.path.isdir(category_path): continue

  # Create subdirectories for each class in train and val files
  os.makedirs(os.path.join(train_dir, category), exist_ok=True)
  os.makedirs(os.path.join(val_dir, category), exist_ok=True)
  os.makedirs(os.path.join(test_dir, category), exist_ok=True)

  # Get all images in the category to split
  images = os.listdir(category_path)
  random.seed(seed)
  random.shuffle(images)

  # Find indices of image to split
  total_images = len(images)
  train_images_index = int(train_ratio * total_images)
  val_images_index = int((train_ratio + val_ratio) * total_images)

  # Split data by indices
  train_images = images[:train_images_index]
  val_images = images[train_images_index:val_images_index]
  test_images = images[val_images_index:]

  # Move images to train and val directories with folders as original file:
  for image in train_images:
    shutil.copy(os.path.join(category_path, image), os.path.join(train_dir, category))

  for image in val_images:
    shutil.copy(os.path.join(category_path, image), os.path.join(val_dir, category))

  for image in test_images:
    shutil.copy(os.path.join(category_path, image), os.path.join(test_dir, category))

In [3]:
from tensorflow import keras

img_height, img_width = 227, 227
batch_size = 128

# Create Datasets
train_generator = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    seed=seed
)

val_generator = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    seed=seed
)

test_generator = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False
)

class_names = train_generator.class_names
num_classes = len(class_names)

AUTOTUNE = tf.data.AUTOTUNE
train_generator = train_generator.prefetch(buffer_size=AUTOTUNE)
val_generator = val_generator.prefetch(buffer_size=AUTOTUNE)
test_generator = test_generator.prefetch(buffer_size=AUTOTUNE)

Found 4826 files belonging to 8 classes.
Found 1034 files belonging to 8 classes.
Found 1039 files belonging to 8 classes.


In [46]:
# Define AlexNet-like Architecture
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout

model = Sequential([
    Input(shape=(img_height, img_width, 3)),

    Conv2D(filters=16, kernel_size=11, strides=4, activation='relu'),
    MaxPooling2D(pool_size=3, strides=2),
    BatchNormalization(),

    Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=3, strides=2),
    BatchNormalization(),

    Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=3, strides=2),
    BatchNormalization(),

    Flatten(),
    Dense(units=256, activation='relu'),
    Dropout(0.5),
    Dense(units=256, activation='relu'),
    Dropout(0.5),

    Dense(units=num_classes, activation='linear')
])

In [47]:
model.compile(
    optimizer= tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

model.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_47 (Conv2D)                   │ (None, 55, 55, 16)          │           5,824 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_40 (MaxPooling2D)      │ (None, 27, 27, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_30               │ (None, 27, 27, 16)          │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_48 (Conv2D)                   │ (None, 27, 27, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_41 (MaxPooling2D)      │ (None, 13, 13, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_31               │ (None, 13, 13, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_49 (Conv2D)                   │ (None, 13, 13, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_42 (MaxPooling2D)      │ (None, 6, 6, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_32               │ (None, 6, 6, 64)            │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_12 (Flatten)                 │ (None, 2304)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_35 (Dense)                     │ (None, 256)                 │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_23 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_36 (Dense)                     │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_24 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_37 (Dense)                     │ (None, 8)                   │           2,056 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 687,336 (2.62 MB)

 Trainable params: 687,112 (2.62 MB)

 Non-trainable params: 224 (896.00 B)

In [48]:
# Train the Model
epochs = 20
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,
)

Epoch 1/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 12s 254ms/step - accuracy: 0.4673 - loss: 1.8552 - val_accuracy: 0.3801 - val_loss: 2.7197
Epoch 2/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 10s 246ms/step - accuracy: 0.7731 - loss: 0.6852 - val_accuracy: 0.3443 - val_loss: 3.0037
Epoch 3/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 10s 246ms/step - accuracy: 0.8317 - loss: 0.4767 - val_accuracy: 0.6103 - val_loss: 1.4487
Epoch 4/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 10s 252ms/step - accuracy: 0.8735 - loss: 0.3610 - val_accuracy: 0.6838 - val_loss: 0.9092
Epoch 5/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 10s 254ms/step - accuracy: 0.9092 - loss: 0.2353 - val_accuracy: 0.7360 - val_loss: 0.8264
Epoch 6/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 10s 253ms/step - accuracy: 0.9236 - loss: 0.2136 - val_accuracy: 0.5484 - val_loss: 2.1070
Epoch 7/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 10s 250ms/step - accuracy: 0.9300 - loss: 0.1934 - val_accuracy: 0.6702 - val_loss: 1.5778
Epoch 8/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 9s 246ms/step - accuracy: 0.9335 - loss: 0.1787 - val_accur

In [49]:
# Evaluate on Test Set
import numpy as np
from sklearn.metrics import classification_report

loss, accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {accuracy:.5f}")

# Classification Report
y_true = []
y_pred = []

for images, labels in test_generator:
    preds = model.predict(images, verbose=0)
    preds = np.argmax(preds, axis=1)
    y_pred.extend(preds.tolist())
    y_true.extend(labels.numpy().tolist())

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names, zero_division=0))

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - accuracy: 0.9346 - loss: 0.2189
Test accuracy: 0.94033
Classification Report:
              precision    recall  f1-score   support

    airplane       0.98      0.96      0.97       110
         car       1.00      0.97      0.99       146
         cat       0.81      0.86      0.83       133
         dog       0.78      0.76      0.77       106
      flower       0.94      0.94      0.94       127
       fruit       1.00      1.00      1.00       150
   motorbike       0.98      1.00      0.99       119
      person       1.00      0.98      0.99       148

    accuracy                           0.94      1039
   macro avg       0.94      0.94      0.94      1039
weighted avg       0.94      0.94      0.94      1039

